# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
#Type your code here
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skopt import BayesSearchCV

# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
# Lengkapi kode berikut
df = pd.read_csv("data_clustering_inverse.csv")

In [3]:
# Tampilkan 5 baris pertama dengan function head.
df.head(5)

,TransactionAmount,TransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,TransactionAmountCategory,Target
0,14.09,2023-04-11 16:29:14,Debit,San Diego,ATM,70.0,Doctor,81.0,1.0,5112.21,2024-11-04 08:08:08,0,1
1,376.24,2023-06-27 16:44:19,Debit,Houston,ATM,68.0,Doctor,141.0,1.0,13758.91,2024-11-04 08:09:35,1,2
2,126.29,2023-07-10 18:16:08,Debit,Mesa,Online,19.0,Student,56.0,1.0,1122.35,2024-11-04 08:07:04,0,1
3,184.50,2023-05-05 16:32:11,Debit,Raleigh,Online,26.0,Student,25.0,1.0,8569.06,2024-11-04 08:09:06,0,1
4,13.45,2023-10-16 17:51:24,Credit,Atlanta,Online,45.0,Student,198.0,1.0,7429.40,2024-11-04 08:06:39,0,2


In [4]:
# Cek Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2514 entries, 0 to 2513
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   TransactionAmount          2514 non-null   float64
 1   TransactionDate            2514 non-null   object 
 2   TransactionType            2484 non-null   object 
 3   Location                   2484 non-null   object 
 4   Channel                    2487 non-null   object 
 5   CustomerAge                2514 non-null   float64
 6   CustomerOccupation         2491 non-null   object 
 7   TransactionDuration        2514 non-null   float64
 8   LoginAttempts              2514 non-null   float64
 9   AccountBalance             2514 non-null   float64
 10  PreviousTransactionDate    2514 non-null   object 
 11  TransactionAmountCategory  2514 non-null   int64  
 12  Target                     2514 non-null   int64  
dtypes: float64(5), int64(2), object(6)
memory usage:

In [5]:
# Cek Missing Values
df.isnull().sum()

TransactionAmount             0
TransactionDate               0
TransactionType              30
Location                     30
Channel                      27
CustomerAge                   0
CustomerOccupation           23
TransactionDuration           0
LoginAttempts                 0
AccountBalance                0
PreviousTransactionDate       0
TransactionAmountCategory     0
Target                        0
dtype: int64

In [6]:
# Tangani Missing Values dengan Mode untuk feature Kategorik

modus_balance = df.select_dtypes(include='object').mode().iloc[0]
df.fillna(modus_balance, inplace=True)

# Lihat hasil akhir
missing_values = df.isnull().sum()
print(f"Missing Values now:\n {missing_values}")

Missing Values now:
 TransactionAmount            0
TransactionDate              0
TransactionType              0
Location                     0
Channel                      0
CustomerAge                  0
CustomerOccupation           0
TransactionDuration          0
LoginAttempts                0
AccountBalance               0
PreviousTransactionDate      0
TransactionAmountCategory    0
Target                       0
dtype: int64


In [7]:
# Drop Feature tak digunakan (TransactionDate dan PreviousTransactionDate)
df = df.drop(columns=['TransactionDate', 'PreviousTransactionDate'])

In [8]:
df.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionAmountCategory,Target
0,14.09,Debit,San Diego,ATM,70.0,Doctor,81.0,1.0,5112.21,0,1
1,376.24,Debit,Houston,ATM,68.0,Doctor,141.0,1.0,13758.91,1,2
2,126.29,Debit,Mesa,Online,19.0,Student,56.0,1.0,1122.35,0,1
3,184.50,Debit,Raleigh,Online,26.0,Student,25.0,1.0,8569.06,0,1
4,13.45,Credit,Atlanta,Online,45.0,Student,198.0,1.0,7429.40,0,2


In [9]:
# Encoding Feature Kategorik
categoric_cols = ['TransactionType', 'Location', 'Channel', 'CustomerOccupation']

label_encoder_dict = {}

# Encoding
for col in categoric_cols:
    label_encoder = LabelEncoder()
    df[col] = label_encoder.fit_transform(df[col])

    #simpan objek encoder terlatih ke kamus agar dapat diinverse
    label_encoder_dict[col] = label_encoder

df.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,TransactionAmountCategory,Target
0,14.09,1,36,0,70.0,0,81.0,1.0,5112.21,0,1
1,376.24,1,15,0,68.0,0,141.0,1.0,13758.91,1,2
2,126.29,1,23,2,19.0,3,56.0,1.0,1122.35,0,1
3,184.50,1,33,2,26.0,3,25.0,1.0,8569.06,0,1
4,13.45,0,1,2,45.0,3,198.0,1.0,7429.40,0,2


# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [10]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.
X = df.drop(columns=['Target'])
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [11]:
# Buatlah model klasifikasi menggunakan Decision Tree
dt_model = DecisionTreeClassifier().fit(X_train, y_train)

In [12]:
# Model Evaluation
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Print hasil
    print(f"========= Evaluasi Model {model} ==========")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1_Score: {f1:4f}")
    print("==================================================\n")

evaluate_model(dt_model, X_test, y_test)

========= Evaluasi Model DecisionTreeClassifier() ==========
Accuracy: 0.9821
Precision: 0.9826
Recall: 0.9821
F1_Score: 0.982151



Evaluasi model menunjukkan hasil yang sangat bagus (mendekati 1).

In [13]:
# Menyimpan Model
import joblib
joblib.dump(dt_model, 'decision_tree_model.h5')

['decision_tree_model.h5']

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [14]:
# Melatih model menggunakan algoritma klasifikasi scikit-learn selain Decision Tree.
rf = RandomForestClassifier().fit(X_train, y_train)
nb = GaussianNB().fit(X_train, y_train)
knn = KNeighborsClassifier().fit(X_train, y_train)

In [15]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.
evaluate_model(rf, X_test, y_test)
evaluate_model(nb, X_test, y_test)
evaluate_model(knn, X_test, y_test)

========= Evaluasi Model RandomForestClassifier() ==========
Accuracy: 0.9801
Precision: 0.9811
Recall: 0.9801
F1_Score: 0.980218

========= Evaluasi Model GaussianNB() ==========
Accuracy: 0.9225
Precision: 0.9359
Recall: 0.9225
F1_Score: 0.924533

========= Evaluasi Model KNeighborsClassifier() ==========
Accuracy: 0.8330
Precision: 0.8330
Recall: 0.8330
F1_Score: 0.830076



**Summary :**
Dilihat dari hasil evaluasi, ternyata Decission Tree menghasilkan model dengan metrik evaluasi tertinggi. <br>
1. Decision Tree
2. Random Forest
3. Naive Bayes
4. KNN

In [16]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
import joblib
joblib.dump(rf, 'explore_RandomForest_classification.h5')
joblib.dump(nb, 'explore_NaiveBayes_classification.h5')
joblib.dump(knn, 'explore_KNN_classification.h5')

['explore_KNN_classification.h5']

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [21]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
# Lakukan dalam satu cell ini saja.

# HyperparameterTuning dengan Bayesian Optimization
#Definisi ruang pencarian
param_space = {
    'criterion': ['gini', 'entropy'],
    'max_depth': (1, 10),
    'min_samples_split': (2, 20),
    'min_samples_leaf': (1, 20),
    'max_features': ['sqrt', 'log2', None],
}

# Inisialisasi BayesSearchCV
bayes_search = BayesSearchCV(estimator=dt_model, search_spaces=param_space, n_iter=32, cv=3, n_jobs=-1, verbose=2, random_state=42)
bayes_search.fit(X_train, y_train)

print(f"Best parameters (Bayesian Optimization): {bayes_search.best_params_}")
best_dt_bayes = bayes_search.best_estimator_

bayes_search_score = best_dt_bayes.score(X_test, y_test)
print(f"Accuracy setelah Bayesian Optimization: {bayes_search_score:.2f}")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

In [22]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.

evaluate_model(best_dt_bayes, X_test, y_test)

========= Evaluasi Model DecisionTreeClassifier(max_depth=10, min_samples_split=10) ==========
Accuracy: 0.9881
Precision: 0.9884
Recall: 0.9881
F1_Score: 0.988033



**Summary :**

Dari hasil tuning dengan Bayesian, dapat dilihat jika evaluasi akurasi, presisi, recall, dan F-1 Score jadi lebih tinggi dari evaluasi model sebelum tuning.

In [23]:
# Menyimpan Model hasil tuning
import joblib
joblib.dump(best_dt_bayes, 'tuning_classification.h5')

['tuning_classification.h5']

End of Code